## Deliverable 3. Create a Travel Itinerary Map.

In [9]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
import json
# Import API key
from config1 import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
# vacation_df = pd.read_csv("Vacation_Search/WeatherPy_vacation.csv")
# vacation_df.head()

wp_data_to_load = "WeatherPy_vacation.csv"
wp_data_df = pd.read_csv(wp_data_to_load)
wp_data_df.head()

,City_ID,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,6,Kapaa,US,83.12,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
1,8,Hilo,US,81.81,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
2,14,Rikitea,PF,76.55,light rain,-23.1203,-134.9692,Pension Maro'i
3,19,Avarua,CK,82.45,broken clouds,-21.2078,-159.7750,Paradise Inn
4,20,Atuona,PF,80.37,scattered clouds,-9.8000,-139.0333,Villa Enata


In [3]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Description</dt><dd>{Description}</dd>
</dl>
"""


# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
wp_info = [info_box_template.format(**row) for index, row in wp_data_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = wp_data_df[["Lat", "Lng"]]


In [4]:
# https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html#directions-layer
# 4a. Add a marker layer for each city to the map.
max_temp = wp_data_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=wp_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 4b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))

In [20]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = wp_data_df.loc[(wp_data_df["City"]== "Houston")]
vacation_end = wp_data_df.loc[(wp_data_df["City"]==  "Houston")]
vacation_stop1 = wp_data_df.loc[(wp_data_df["City"]==  "Farmington")]
vacation_stop2 = wp_data_df.loc[(wp_data_df["City"]==  "Laguna")] 
vacation_stop3 = wp_data_df.loc[(wp_data_df["City"]==  "Scottsbluff")] 

In [21]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and 
#list indexing.
start = vacation_start["Lat"].to_numpy()[0], vacation_start["Lng"].to_numpy()[0]
end = vacation_end["Lat"].to_numpy()[0], vacation_end["Lng"].to_numpy()[0]
print(f'Start: {start}, End: {end}')
stop1 = vacation_stop1["Lat"].to_numpy()[0], vacation_stop1["Lng"].to_numpy()[0]
stop2 = vacation_stop2["Lat"].to_numpy()[0], vacation_stop2["Lng"].to_numpy()[0]
stop3 = vacation_stop3["Lat"].to_numpy()[0], vacation_stop3["Lng"].to_numpy()[0]
print(f'Stop 1: {stop1}, Stop 2: {stop2} , Stop 3: {stop3}')


Start: (29.7633, -95.3633), End: (29.7633, -95.3633)
Stop 1: (36.7281, -108.2187), Stop 2: (38.421, -121.4238) , Stop 3: (41.8666, -103.6672)


In [22]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
fig = gmaps.figure()
city_itinerary = gmaps.directions_layer(
        start, end, waypoints=[stop1, stop2, stop3],
        travel_mode='DRIVING')

fig.add_layer(city_itinerary)
fig


Figure(layout=FigureLayout(height='420px'))

In [24]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3],ignore_index=True)
itinerary_df
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html

,City_ID,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,57,Houston,US,84.34,broken clouds,29.7633,-95.3633,"Hotel ICON, Autograph Collection"
1,117,Farmington,US,79.86,clear sky,36.7281,-108.2187,Courtyard by Marriott Farmington
2,647,Laguna,US,84.90,clear sky,38.4210,-121.4238,Holiday Inn Express & Suites Elk Grove Central...
3,670,Scottsbluff,US,75.78,scattered clouds,41.8666,-103.6672,Hampton Inn & Suites Scottsbluff-Conference Ce...


In [27]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Description</dt><dd>{Description}</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [30]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure()
#city_itinerary = gmaps.directions_layer(
        #start, end, waypoints=[stop1, stop2, stop3],
        #travel_mode='DRIVING')
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
#fig.add_layer(city_itinerary)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))